In [ ]:
#  Most of these libraries are included in the Anaconda distribution of Python.  Download necessary libraries through the Anaconda prompt with pip install or conda install.  

import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from thermo.chemical import Chemical
import tkinter
from tkinter import *
from tkinter import filedialog
import os

In [4]:
#  this cell opens a dialog to load the template spreadsheet

root = tkinter.Tk()
request_load_file = filedialog.askopenfile(initialdir=os.getcwd(),filetypes=[('Excel Files', '*.xlsx'),('CSV Files', '*.csv')])
if request_load_file:
      filepath = os.path.abspath(request_load_file.name)
root.destroy()
if filepath[-5:]=='.xlsx':
    df_inventory = pd.read_excel(filepath)
elif filepath[-4:]=='.csv':
    df_inventory = pd.read_csv(filepath)
df_inventory.head()

,Entry,Compound,CAS,Supplier,Initial Amount,Current Amount,Location,Storage Code,Hazard Class,"Bottle type (glass, plastic, etc.)",Date received,Barcode,Room,Expiration date?,Last Checked Date,Additional Info,Comments/Updates
0,DH607A-2,Bromobenzene,108-86-1,Repackaged,120mL,120mL,Flammable 3,NEUT A-Z,3,Glass,2017-09-29 00:00:00,C006668,DH607A,NaT,AJC 8/11/21,NaN,NaN
1,DH607A-3,2-methyl-2-butene,513-35-9,Sigma Aldrich,50mL,50mL,Flammable 3,NEUT A-Z,3,Glass,2015-07-27 00:00:00,C008259,DH607A,NaT,AJC 8/11/21,NaN,NaN
2,DH607A-4,1-bromo-3-fluorobenzene,1073-06-9,AK Scientific,25g,25g,Flammable 3,NEUT A-Z,3,Glass,2015-07-06 00:00:00,C008279,DH607A,NaT,AJC 8/11/21,NaN,NaN
3,DH607A-5,(1R)-(+)-α-Pinene,7785-70-8,TCI,25 mL,25 mL,Flammable 3,NEUT A-Z,3,Glass,2016-06-13 00:00:00,C009130,DH607A,NaT,AJC 8/11/21,NaN,NaN
4,DH607A-7,Benzene-d6,71-43-2,Cambridge Isotope Lab.,10 g,3 g,Flammable 3,NMR Solvent,3,Glass,2017-11-10 00:00:00,C036553,DH607A,NaT,AJC 8/11/21,NaN,NaN


In [38]:
filepath[-5:]

'.xlsx'

In [37]:
filepath

'C:\\Users\\chadm\\Desktop\\Miller Conrad Lab\\Inventory\\MillerConradLab_Inventory_11012021_forChad.xlsx'

In [8]:
#  Note: The loaded spreadsheet must contain the CAS numbers of the chemicals in a column named "CAS"
#  This cell creates 2 new columns in the dataframe and cross-references PubChem IDs with given CAS numbers

df_inventory['PubChem ID']=np.nan
df_inventory['GHS Codes']=np.nan
for i in df_inventory['CAS']:
    try:
        chem = Chemical(f'{i}')
        df_inventory.loc[df_inventory['CAS']==i,'PubChem ID'] = chem.PubChem
    except Exception:
        pass

In [14]:
#  This is a test run of the code that produces GHS codes.  Run this and ensure the output is readable before you start the loop in the next window.

test_chem_id = df_inventory['PubChem ID'].dropna()[0]
result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(test_chem_id)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
soup = BeautifulSoup(result.text,'lxml').text
if len(soup) > 90:
    pattern_hits = [m.start() for m in re.finditer(r'"H\d\d\d', soup)]
    ghs_codes_set = set()
    for i in range(0,len(pattern_hits)):
        phrase_start = pattern_hits[i]
        phrase_end = soup[phrase_start+1:].find('"')
        ghs_codes_set.add(soup[phrase_start+1:phrase_start+5])
    ghs_codes_list = list()
    for c in ghs_codes_set:
        ghs_start = soup.find(c)
        ghs_end = soup[soup.find(c)+1:].find('"')+1
        ghs_codes_list.append(soup[ghs_start:ghs_start+ghs_end])
    joined_ghs = ' --- '.join(ghs_codes_list)
    print(joined_ghs)
    #df_inventory.loc[df_inventory['PubChem ID']==chem_id,'GHS Codes'] = joined_ghs
else:
    pass

H331: Toxic if inhaled [Danger Acute toxicity, inhalation] --- H315: Causes skin irritation [Warning Skin corrosion/irritation] --- H226: Flammable liquid and vapor [Warning Flammable liquids] --- H411: Toxic to aquatic life with long lasting effects [Hazardous to the aquatic environment, long-term hazard] --- H373: Causes damage to organs through prolonged or repeated exposure [Warning Specific target organ toxicity, repeated exposure]


In [234]:
#  This cell will loop over the PubChem IDs and request GHS data from NCBI.  This could have a runtime of several minutes depending on your hardware and internet connection.

for chem_id in set(df_inventory['PubChem ID'].dropna()):
    result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(chem_id)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
    soup = BeautifulSoup(result.text,'lxml').text
    if len(soup) > 90:
        pattern_hits = [m.start() for m in re.finditer(r'"H\d\d\d', soup)]
        ghs_codes_set = set()
        for i in range(0,len(pattern_hits)):
            phrase_start = pattern_hits[i]
            phrase_end = soup[phrase_start+1:].find('"')
            ghs_codes_set.add(soup[phrase_start+1:phrase_start+5])
        ghs_codes_list = list()
        for c in ghs_codes_set:
            ghs_start = soup.find(c)
            ghs_end = soup[soup.find(c)+1:].find('"')+1
            ghs_codes_list.append(soup[ghs_start:ghs_start+ghs_end])
        joined_ghs = ' --- '.join(ghs_codes_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'GHS Codes'] = joined_ghs
    else:
        pass

In [36]:
# input your filename as a string in this cell.  Files will be saved in '.xlsx' format by default.

save_as_filename = ''

Save file as: test55


In [32]:
#  If you prefer .csv format, simply replace '.xlsx' below with '.csv'
#  Edited spreadsheet will be saved in the same location as the origional template

df_inventory.to_excel(filepath[:filepath.rfind('\\')+1]+save_as_filename+'.xlsx')